In [3]:
import pandas as pd
from meteostat import Point, Daily
from datetime import datetime, timedelta
import requests
import ssl
from geopy.distance import geodesic

ssl._create_default_https_context = ssl._create_unverified_context



In [4]:



completed_orders= pd.read_csv('../data/processed/nb_cleaned.csv')
delivery_requests = pd.read_csv('../data/processed/location_cleaned.csv')



In [5]:
def parse_coordinates(address):
    """
    Parse the address to extract latitude and longitude.
    :param address: String in the format "latitude, longitude"
    :return: Tuple containing (latitude, longitude)
    """
    # Clean up the address string
    address = address.strip()  # Remove leading and trailing whitespace
    lat, lng = map(float, address.split(','))  # Split and convert to float
    return lat, lng


# Apply the function to the Trip Origin and Trip Destination columns
completed_orders[['Origin Lat', 'Origin Lng']] = completed_orders['Trip Origin'].apply(parse_coordinates).apply(pd.Series)
completed_orders[['Destination Lat', 'Destination Lng']] = completed_orders['Trip Destination'].apply(parse_coordinates).apply(pd.Series)


In [6]:

def calculate_distance(row):
    origin = (row['Origin Lat'], row['Origin Lng'])
    destination = (row['Destination Lat'], row['Destination Lng'])
    return geodesic(origin, destination).kilometers

completed_orders['trip_distance'] = completed_orders.apply(calculate_distance, axis=1)

print(completed_orders.head())


   Unnamed: 0  Trip ID                         Trip Origin  \
0           0   391996  6.508813001668548,3.37740316890347   
1           1   391997                 6.4316714,3.4555375   
2           2   391998         6.631679399999999,3.3388976   
3           3   391999         6.572757200000001,3.3677082   
4           4   392001                 6.6010417,3.2766339   

                  Trip Destination      Trip Start Time        Trip End Time  \
0      6.650969799999999,3.3450307  2021-07-01 07:28:04  2021-07-01 07:29:37   
1  6.4280814653326,3.4721885847586  2021-07-01 06:38:04  2021-07-01 07:07:28   
2      6.508324099999999,3.3590397  2021-07-01 06:21:02  2021-07-01 07:02:23   
3      6.584881099999999,3.3614073  2021-07-01 07:16:07  2021-07-01 07:29:42   
4              6.4501069,3.3916154  2021-07-01 09:30:59  2021-07-01 09:34:36   

   Origin Lat  Origin Lng  Destination Lat  Destination Lng  trip_distance  
0    6.508813    3.377403         6.650970         3.345031      16.1

In [7]:
completed_orders.to_csv('../data/processed/nb_cleaned.csv', index=False)


Calculating the distance between th driver location and the trip origin for completed trips

In [95]:
completed_orders.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70047 entries, 0 to 70046
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        70047 non-null  int64  
 1   Trip ID           70047 non-null  int64  
 2   Trip Origin       70047 non-null  object 
 3   Trip Destination  70047 non-null  object 
 4   Trip Start Time   70047 non-null  object 
 5   Trip End Time     70047 non-null  object 
 6   Origin Lat        70047 non-null  float64
 7   Origin Lng        70047 non-null  float64
 8   Destination Lat   70047 non-null  float64
 9   Destination Lng   70047 non-null  float64
 10  trip_distance     70047 non-null  float64
dtypes: float64(5), int64(2), object(4)
memory usage: 5.9+ MB


In [8]:
completed_orders['Trip Origin']
completed_orders[['Origin Lat', 'Origin Lng']] = completed_orders['Trip Origin'].apply(
    lambda x: pd.Series(map(float, x.split(',')))
)


In [9]:
completed_orders.head()

,Unnamed: 0,Trip ID,Trip Origin,Trip Destination,Trip Start Time,Trip End Time,Origin Lat,Origin Lng,Destination Lat,Destination Lng,trip_distance
0,0,391996,"6.508813001668548,3.37740316890347","6.650969799999999,3.3450307",2021-07-01 07:28:04,2021-07-01 07:29:37,6.508813,3.377403,6.650970,3.345031,16.123451
1,1,391997,"6.4316714,3.4555375","6.4280814653326,3.4721885847586",2021-07-01 06:38:04,2021-07-01 07:07:28,6.431671,3.455538,6.428081,3.472189,1.884305
2,2,391998,"6.631679399999999,3.3388976","6.508324099999999,3.3590397",2021-07-01 06:21:02,2021-07-01 07:02:23,6.631679,3.338898,6.508324,3.359040,13.822393
3,3,391999,"6.572757200000001,3.3677082","6.584881099999999,3.3614073",2021-07-01 07:16:07,2021-07-01 07:29:42,6.572757,3.367708,6.584881,3.361407,1.511034
4,4,392001,"6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36,6.601042,3.276634,6.450107,3.391615,20.984319


In [14]:
completed_orders = pd.read_csv('../data/processed/nb_cleaned.csv')
merged_df = delivery_requests.merge(
    completed_orders[['Trip ID', 'Origin Lat']], 
    left_on='order_id', 
    right_on='Trip ID', 
    how='left'
)


In [17]:
# columns_to_drop = ['Unnamed: 0', 'Trip Origin_x', 'origin_lat', 'origin_lng', 'Trip ID_y', 'Trip Origin_y', 'Trip ID_x']
# cleaned_df = merged_df.drop(columns=columns_to_drop)
merged_df = merged_df.merge(
     completed_orders[['Trip ID', 'Origin Lng', 'trip_distance']], 
    left_on='order_id', 
    right_on='Trip ID', 
    how='left'
)


In [21]:
merged_df.head(2)
columns_to_drop = ['Trip ID_x']
merged_df = merged_df.drop(columns=columns_to_drop)

In [23]:
merged_df.head(2)
merged_df.to_csv('safe.csv')

In [29]:


print(merged_df.isna().sum())
merged_df = merged_df.dropna(subset=['lat', 'lng', 'Origin Lat', 'Origin Lng'])



order_id            0
driver_id           0
driver_action       0
lat                 0
lng                 0
Origin Lat       2022
Origin Lng       2022
trip_distance    2022
dtype: int64


In [35]:
def calculate_driver_to_order_distance(row):
    driver_location = (row['lat'], row['lng'])
    order_location = (row['Origin Lat'], row['Origin Lng'])
    return geodesic(driver_location, order_location).kilometers


In [36]:
merged_df['driver_to_order_distance'] = merged_df.apply(calculate_driver_to_order_distance, axis=1)


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [39]:
merged_df.head()
# merged_df.to_csv('safe.csv')


,order_id,driver_id,driver_action,lat,lng,Origin Lat,Origin Lng,trip_distance,driver_to_order_distance
0,392001,243828,accepted,6.602207,3.270465,6.601042,3.276634,20.984319,0.694264
1,392001,243588,rejected,6.592097,3.287445,6.601042,3.276634,20.984319,1.551694
2,392001,243830,rejected,6.596133,3.281784,6.601042,3.276634,20.984319,0.786777
3,392001,243539,rejected,6.596142,3.280526,6.601042,3.276634,20.984319,0.692054
4,392001,171653,rejected,6.609232,3.288800,6.601042,3.276634,20.984319,1.621848


In [ ]:
driver_location = (6.602207,3.270465 )
order_location = (	6.601042, 3.276634)

# Calculate the distance in meters
distance = geodesic(driver_location, order_location).meters
print(distance)


694.2656653179304


In [75]:
delivery_requests.to_csv('../data/external/delivery_requests_with_distances.csv', index=False)


KeyboardInterrupt: 

In [74]:
delivery_requests = pd.read_csv('delivery_requests_with_distances.csv')
delivery_requests.head(3)

FileNotFoundError: [Errno 2] No such file or directory: 'delivery_requests_with_distances.csv'

In [ ]:
delivery_requests.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1557740 entries, 0 to 1557739
Data columns (total 11 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   Unnamed: 0                1557740 non-null  int64  
 1   id                        1557740 non-null  int64  
 2   order_id                  1557740 non-null  int64  
 3   driver_id                 1557740 non-null  int64  
 4   driver_action             1557740 non-null  object 
 5   lat                       1557740 non-null  float64
 6   lng                       1557740 non-null  float64
 7   driver_to_order_distance  1555718 non-null  float64
 8   Trip ID                   1555718 non-null  float64
 9   Origin Lat                1555718 non-null  float64
 10  Origin Lng                1555718 non-null  float64
dtypes: float64(6), int64(4), object(1)
memory usage: 130.7+ MB


In [43]:
# completed_orders.info()
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1555718 entries, 0 to 1557739
Data columns (total 9 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   order_id                  1555718 non-null  int64  
 1   driver_id                 1555718 non-null  int64  
 2   driver_action             1555718 non-null  object 
 3   lat                       1555718 non-null  float64
 4   lng                       1555718 non-null  float64
 5   Origin Lat                1555718 non-null  float64
 6   Origin Lng                1555718 non-null  float64
 7   trip_distance             1555718 non-null  float64
 8   driver_to_order_distance  1555718 non-null  float64
dtypes: float64(6), int64(2), object(1)
memory usage: 150.9+ MB


In [46]:
completed_orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70047 entries, 0 to 70046
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        70047 non-null  int64  
 1   Trip ID           70047 non-null  int64  
 2   Trip Origin       70047 non-null  object 
 3   Trip Destination  70047 non-null  object 
 4   Trip Start Time   70047 non-null  object 
 5   Trip End Time     70047 non-null  object 
 6   Origin Lat        70047 non-null  float64
 7   Origin Lng        70047 non-null  float64
 8   Destination Lat   70047 non-null  float64
 9   Destination Lng   70047 non-null  float64
 10  trip_distance     70047 non-null  float64
dtypes: float64(5), int64(2), object(4)
memory usage: 5.9+ MB


In [57]:
completed_orders.rename(columns={'Trip ID': 'order_id'}, inplace=True)
merged_df = pd.merge(merged_df, completed_orders[['order_id', 'Trip Start Time', 'Trip End Time']], on='order_id', how='left')
print(merged_df.head())


   order_id  driver_id driver_action       lat       lng  Origin Lat  \
0    392001     243828      accepted  6.602207  3.270465    6.601042   
1    392001     243588      rejected  6.592097  3.287445    6.601042   
2    392001     243830      rejected  6.596133  3.281784    6.601042   
3    392001     243539      rejected  6.596142  3.280526    6.601042   
4    392001     171653      rejected  6.609232  3.288800    6.601042   

   Origin Lng  trip_distance  driver_to_order_distance      Trip Start Time  \
0    3.276634      20.984319                  0.694264  2021-07-01 09:30:59   
1    3.276634       0.000000                  1.551694  2021-07-01 09:30:59   
2    3.276634       0.000000                  0.786777  2021-07-01 09:30:59   
3    3.276634       0.000000                  0.692054  2021-07-01 09:30:59   
4    3.276634       0.000000                  1.621848  2021-07-01 09:30:59   

         Trip End Time  
0  2021-07-01 09:34:36  
1  2021-07-01 09:34:36  
2  2021-07-01 09:

In [58]:
merged_df.loc[merged_df['driver_action'] == 'rejected', ['Trip Start Time', 'Trip End Time', 'trip_distance']] = 0
# merged_df.rename(columns={'Trip Start Time_x': 'Trip Start Time'}, inplace=True)
# merged_df.rename(columns={'Trip End Time_x': 'Trip End Time'}, inplace=True)
# columns_to_drop = [ 'Trip End Time_y', 'Trip Start Time_y']
# merged_df = merged_df.drop(columns=columns_to_drop)

merged_df.head(2)

,order_id,driver_id,driver_action,lat,lng,Origin Lat,Origin Lng,trip_distance,driver_to_order_distance,Trip Start Time,Trip End Time
0,392001,243828,accepted,6.602207,3.270465,6.601042,3.276634,20.984319,0.694264,2021-07-01 09:30:59,2021-07-01 09:34:36
1,392001,243588,rejected,6.592097,3.287445,6.601042,3.276634,0.000000,1.551694,0,0


In [59]:
merged_df['Trip Start Time'] = pd.to_datetime(merged_df['Trip Start Time'])
merged_df['Trip End Time'] = pd.to_datetime(merged_df['Trip End Time'])


In [61]:
merged_df['time_taken'] = (merged_df['Trip End Time'] - merged_df['Trip Start Time']).dt.total_seconds()
merged_df['speed'] = merged_df['trip_distance'] / merged_df['time_taken']
merged_df.head(2)

,order_id,driver_id,driver_action,lat,lng,Origin Lat,Origin Lng,trip_distance,driver_to_order_distance,Trip Start Time,Trip End Time,time_taken,speed
0,392001,243828,accepted,6.602207,3.270465,6.601042,3.276634,20.984319,0.694264,2021-07-01 09:30:59,2021-07-01 09:34:36,217.0,0.096702
1,392001,243588,rejected,6.592097,3.287445,6.601042,3.276634,0.000000,1.551694,1970-01-01 00:00:00,1970-01-01 00:00:00,0.0,NaN


In [62]:
merged_df.to_csv('safe.csv')

Holidays and weekdays

In [82]:
import holidays
from datetime import date
merged_df = pd.read_csv('safe.csv')
merged_df.head(2)


,Unnamed: 0,order_id,driver_id,driver_action,lat,lng,Origin Lat,Origin Lng,trip_distance,driver_to_order_distance,Trip Start Time,Trip End Time,time_taken,speed
0,0,392001,243828,accepted,6.602207,3.270465,6.601042,3.276634,20.984319,0.694264,2021-07-01 09:30:59,2021-07-01 09:34:36,217.0,0.096702
1,1,392001,243588,rejected,6.592097,3.287445,6.601042,3.276634,0.000000,1.551694,1970-01-01 00:00:00,1970-01-01 00:00:00,0.0,NaN


In [84]:
merged_df['Trip End Time'] = pd.to_datetime(merged_df['Trip End Time']).dt.date 

In [77]:
# ng_holidays = holidays.Nigeria()

# def is_holiday_ng(trip_end_time):

#     return trip_end_time in ng_holidays

In [78]:
# def is_holiday_ng(trip_end_time):
#     return trip_end_time in ng_holidays

# merged_df['is_holiday'] = merged_df['Trip End Time'].apply(is_holiday_ng)


In [79]:
# merged_df = merged_df[merged_df['is_holiday'] == True]
# merged_df.head(2)

,Unnamed: 0,order_id,driver_id,driver_action,lat,lng,Origin Lat,Origin Lng,trip_distance,driver_to_order_distance,Trip Start Time,Trip End Time,time_taken,speed,is_holiday


In [85]:
def classify_weekday_or_weekend(dt):
    if dt.weekday() < 5:  # Monday = 0, Sunday = 6
        return 'Weekday'
    else:
        return 'Weekend'

merged_df['weekday_or_weekend'] = merged_df['Trip End Time'].apply(classify_weekday_or_weekend)

print(merged_df[['Trip End Time', 'weekday_or_weekend']])


        Trip End Time weekday_or_weekend
0          2021-07-01            Weekday
1          1970-01-01            Weekday
2          1970-01-01            Weekday
3          1970-01-01            Weekday
4          1970-01-01            Weekday
...               ...                ...
1555713    1970-01-01            Weekday
1555714    1970-01-01            Weekday
1555715    1970-01-01            Weekday
1555716    1970-01-01            Weekday
1555717    1970-01-01            Weekday

[1555718 rows x 2 columns]


In [87]:
merged_df.to_csv('safe.csv')


In [91]:
from meteostat import Stations, Daily


In [92]:
merged_df['Trip Start Time'] = pd.to_datetime(merged_df['Trip Start Time']).dt.date 

In [93]:
def fetch_weather_data(lat, lon, start_date):
    # Find nearest weather station using Meteostat
    stations = Stations()
    station = stations.nearby(lat, lon).fetch(1)
    
    # Fetch daily weather data for the specified date
    data = Daily(station, start_date)
    data = data.fetch()
    
    return data

In [94]:
precipitation_data = []
for index, row in merged_df.iterrows():
    lat = row['Origin Lat']
    lon = row['Origin Lng']
    trip_start_time = row['Trip Start Time']
    
    # Fetch precipitation data for the specific date of the trip
    weather_data = fetch_weather_data(lat, lon, trip_start_time)
    
    # Extract precipitation information if available
    if not weather_data.empty:
        precipitation = weather_data.iloc[0]['prcp']
    else:
        precipitation = None
    
    precipitation_data.append(precipitation)
